<a href="https://colab.research.google.com/github/ayoni02/AI-Hacktober-MLSA/blob/main/Project_1/Data/Image_scraping_using_fastai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook contains code that can allow you to scrape multiple image types from the duckgo search engine. I exracted it from [the fastai github](https://course.fast.ai/Lessons/lesson2.html).

Head over there to see a much more comprehensive mode of searching

In [ ]:
# use this to download the fastai package
#!pip install -Uqq fastai

In [ ]:
from fastcore.all import *
from fastdownload import download_url
from fastai.vision.all import *
import time
import json

In [ ]:
def search_images(term, max_images=200):
    url = 'https://duckduckgo.com/'
    res = urlread(url,data={'q':term})
    searchObj = re.search(r'vqd=([\d-]+)\&', res)
    requestUrl = url + 'i.js'
    params = dict(l='us-en', o='json', q=term, vqd=searchObj.group(1), f=',,,', p='1', v7exp='a')
    headers = dict( referer='https://duckduckgo.com/' )
    urls,data = set(),{'next':1}
    while len(urls)<max_images and 'next' in data:
        res = urlread(requestUrl, data=params, headers=headers)
        data = json.loads(res) if res else {}
        urls.update(L(data['results']).itemgot('image'))
        requestUrl = url + data['next']
        time.sleep(0.2)
    return L(urls)[:max_images]

In [ ]:
# BSH represent the three folders for each image type and also the name of the image its searching for, you can add or remove from the folder
BSH = 'BUNGALOW BUILDING', 'STOREY BUILDING', 'HIGH RISE BUILDING'
# while path represents the parent folder holding all three the BSH folder
path= Path('Data')

In [ ]:
for o in BSH:
  urls=search_images(o, max_images=100)
  for i in range(100):
    des = (path/o)
    des.mkdir(exist_ok=True, parents=True)
    dest = des/f'{i} {o.lower()}.jpg'
    try:
      #you can turn show progress off soo to reduce the output. I have a bad network so i always want to know how far its progressing
      download_url(urls[i], dest, show_progress=True)
    except:
      continue
    resize_images(path/o, max_size=400, dest=path/o)

In [ ]:
fns = get_image_files(path)
failed = verify_images(fns)
failed

In [ ]:
#This removes anyfailed imgaes from the folders
failed.map(Path.unlink)